In [ ]:
from compute_otu_metrics import read_otu_table, read_taxonomy
from utilities import load_data
otu_taxonomy = read_taxonomy()
site_otu_table = read_otu_table()
site_metadata = load_data()

In [ ]:
level = "KINGDOM"
otu_taxonomy[level] = otu_taxonomy[level].fillna(f"unclassified_{level}").astype(str)
otu_taxonomy[level] = otu_taxonomy[level].replace("Unknown", f"unclassified_{level}")
otu_taxonomy[level].head()
level_site_table = site_otu_table.groupby(otu_taxonomy[level], axis=1).sum() #group otus (otu_table columns) by tax_labels
level_site_table[f'mean_abundance_{level}'] = level_site_table.mean(axis=1)
level_site_table.head()

KINGDOM,Archaea,Bacteria,unclassified_KINGDOM,mean_abundance_KINGDOM
id_site,,,,
6,1880,7186,934,3333.333333
7,1139,7892,969,3333.333333
9,1440,7574,986,3333.333333
10,1119,7422,1459,3333.333333
11,52,8933,1015,3333.333333
